In [1]:
%load_ext autoreload
%autoreload #Use this to reload modules if they are changed on disk while the notebook is running
from __future__ import division
from snmachine import sndata, snfeatures, snclassifier, tsne_plot
import numpy as np
import matplotlib.pyplot as plt
import time, os, pywt,subprocess
from sklearn.decomposition import PCA
from astropy.table import Table,join,vstack
from astropy.io import fits
import sklearn.metrics 
import sncosmo
%matplotlib nbagg
from astropy.table import Column

Neural networks not available in this version of scikit-learn. Neural networks are available from development version 0.18.


## SN numbers with classification code
SN001695   - 33

2457 - 33

2542 - 1

5399 - 2

13481 - 2

13866 - 1 

16742 - 21

17270 - 32

27266 -32

64968 - 23

149147 -21

In [2]:
dataset='spcc'

In [3]:
# WARNING...
#Multinest uses a hardcoded character limit for the output file names. I believe it's a limit of 100 characters
#so avoid making this file path to lengthy if using nested sampling or multinest output file names will be truncated

#Change outdir to somewhere on your computer if you like
outdir=os.path.join('output_%s_no_z' %dataset,'')
out_features=os.path.join(outdir,'features') #Where we save the extracted features to
out_class=os.path.join(outdir,'classifications') #Where we save the classification probabilities and ROC curves
out_int=os.path.join(outdir,'int') #Any intermediate files (such as multinest chains or GP fits)

subprocess.call(['mkdir',outdir])
subprocess.call(['mkdir',out_features])
subprocess.call(['mkdir',out_class])
subprocess.call(['mkdir',out_int])

1

In [4]:
#Data root
rt=os.path.join('SPCC_SUBSET','')

In [5]:
dat=sndata.Dataset(rt)

Reading data...
11 objects read into memory.


In [8]:
read_from_file=True #We can use this flag to quickly rerun from saved features
run_name=os.path.join(out_features,'%s_all' %dataset)

In [14]:
dat.object_names[0]

'DES_SN001695.DAT'

In [15]:
test_table = dat.data['DES_SN001695.DAT'] #get_lightcurve

In [16]:
type_II = []
for i in range(len(dat.data['DES_SN001695.DAT']['flux'])):
    type_II.append(np.random.normal(dat.data['DES_SN001695.DAT']['flux'][i], dat.data['DES_SN001695.DAT']['flux_error'][i], 1000))

In [19]:
type_Ia = []
for i in range(len(dat.data['DES_SN002542.DAT']['flux'])):
    type_Ia.append(np.random.normal(dat.data['DES_SN002542.DAT']['flux'][i], dat.data['DES_SN002542.DAT']['flux_error'][i], 1000))

In [20]:
type_Ibc = []
for i in range(len(dat.data['DES_SN005399.DAT']['flux'])):
    type_Ia.append(np.random.normal(dat.data['DES_SN005399.DAT']['flux'][i], dat.data['DES_SN005399.DAT']['flux_error'][i], 1000))

In [21]:
type_Ia = np.array(type_Ia)
type_Ibc = np.array(type_Ibc)
type_II = np.array(type_II)

In [25]:
test_table.replace_column('flux', type_II[:,0])

In [28]:
#test_table.write('testing.dat',format = 'ascii')

In [24]:
#dat.data['DES_SN001695.DAT']

mjd,filter,flux,flux_error,zp,zpsys
float64,str4,float64,float64,float64,str2
0.0,desg,3.874,4.787,27.5,ab
0.0160000000033,desr,-5.456,5.335,27.5,ab
0.0310000000027,desi,-10.3,5.439,27.5,ab
0.0619999999981,desz,1.439,4.691,27.5,ab
2.0,desg,-0.7297,2.238,27.5,ab
2.14,desr,-2.837,3.03,27.5,ab
2.156,desi,-4.236,4.2,27.5,ab
2.187,desz,-5.086,3.518,27.5,ab
9.984,desg,0.03404,1.702,27.5,ab
